## Loss Analysis

## Table of Contents

1. [Package_Import](#section1)<br>
2. [Data Loading and Description](#section2)
3. [Preprocessing](#section3)

In [ ]:
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from pandas.tseries.offsets import DateOffset
import textwrap
import pyodbc
import plotly
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
#-------------------------------------------------------------------------------------------------------------------------------
import pandas as pd                                                               # Importing for panel data analysis 
pd.set_option('display.max_columns', None)                                        # Unfolding hidden features if the cardinality is high      
pd.set_option('display.max_colwidth', None)                                       # Unfolding the max feature width for better clearity      
pd.set_option('display.max_rows', None)                                           # Unfolding hidden data points if the cardinality is high
pd.set_option('display.float_format', lambda x: '%.2f' % x)                       # To suppress scientific notation over exponential values
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                                # Importing package numpys (For Numerical Python)
#-------------------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                                   # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")     

In [ ]:
pyodbc.drivers()

In [ ]:
connection_string = "DRIVER={SQL Server Native Client 11.0};SERVER=gsc-scpat-sql-001-d.database.windows.net;DATABASE=SC-PAT-DB;UID=SCPAT;PWD=Ecolab@1234"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
try:
    conn = create_engine(connection_url)
except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    

In [ ]:
data_query = textwrap.dedent( """ select
[Period],
[Lag],
[Item],
[Material Type],
[Div],
[Sales Org],
[Cust Facing Loc],
[Origin],
sum(cast([Consensus]as float)) as [Consensus],
sum(Cast([Actual] as float)) as [Actual],
sum(cast([Stat] as float)) as [Stat],
[MD_Error],
[Date],
abs(sum(cast([Consensus]as float)) - sum(Cast([Actual] as float))) as ABS_CONS,
abs(sum(cast([Stat] as float)) - sum(Cast([Actual] as float))) as ABS_STAT
from [FABIAS_Dashboard].[FALossBase_T] GROUP BY
[Period],
[Lag],
[Item],
[Material Type],
[Div],
[Sales Org],
[Cust Facing Loc],
[Origin],
[MD_Error],
[Date]
  """)


In [ ]:
conn

In [ ]:
df = pd.read_sql(data_query,con=conn)

In [ ]:
df.shape

In [ ]:
#fa = pd.read_sql_query('select top(1000) * from [FABIAS_Dashboard].[FALossBase_T]',con=conn)

In [ ]:
#fa['Origin'].unique()

In [ ]:
df['Key'] = df['Item'] +","+ df['Material Type'] +","+ df['Div'] +","+ df['Cust Facing Loc'] +","+ df['Sales Org']+","+df['Lag']


In [ ]:
df.drop_duplicates(subset='Key',inplace=True)

In [ ]:
df.shape

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

In [ ]:
df[df['Key'] == '1826.11R,YSLD,Light,0150023343,4161,Lag 2']

In [ ]:
df['Date'].value_counts().sort_index()

In [ ]:
df[df['Key']=='00-00000145,YSLD,FB_DIV,S4952M000000695700,4952,Lag 1']

In [ ]:
df_group = df.groupby(by=['Key']).sum()[['Actual','Stat','Consensus','ABS_STAT','ABS_CONS']].reset_index()

In [ ]:
df_group.rename(columns={'Actual':'Actual_Sum','Stat':'Stat_Sum','Consensus':'Consensus_Sum',
                     'ABS_STAT':'ABSStat_Sum','ABS_CONS':'ABSCons_Sum'},inplace=True)

In [ ]:
df_group.head()

In [ ]:
df_group.shape

In [ ]:
final_df = pd.merge(df,df_group,on='Key',how='outer')

In [ ]:
final_df.shape

In [ ]:
final_df

### Creating L12M :

In [ ]:
current_Month_StarDate = (datetime.today().date() - pd.offsets.MonthBegin(n=1))

In [ ]:
current_Month_StarDate

In [ ]:
last_12M_date = current_Month_StarDate - pd.DateOffset(months = 12)
last_12M_date

In [ ]:
L12M = df[df['Date'].between(last_12M_date,current_Month_StarDate,inclusive='both')]

In [ ]:
L12M.shape

In [ ]:
L12M=L12M.groupby(by=['Key']).sum()[['Actual','Stat','Consensus','ABS_STAT','ABS_CONS']].reset_index()

In [ ]:
L12M.rename(columns={'Actual':'L12M_Actual_Sum','Stat':'L12M_Stat_Sum','Consensus':'L12M_Consensus_Sum',
                     'ABS_STAT':'L12M_ABSStat_Sum','ABS_CONS':'L12M_ABSCons_Sum'},inplace=True)

In [ ]:
L12M['R12M'] = 'R12M'

In [ ]:
L12M = L12M.groupby(['Key']).value_counts().reset_index(name='L12M_Act_Count')

### Creating L6M Table

In [ ]:
""" 
L6M_query = textwrap.dedent( SELECT [MD_Error]
,[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus]
,sum(cast([Stat]as float)) as [Stat]
,sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [FABIAS_Dashboard].[FALossBaseCal]
where [Date] > Dateadd(Month, -7, getdate()) and [Date] < Dateadd(Month, -1,getdate())
-- WHERE [Item] = '74700PLUS.61R' and [Lag] = 'Lag 2' and [Period] = '202101'
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin] )

"""

In [ ]:
current_Month_StarDate

In [ ]:
last_6M_date = current_Month_StarDate - pd.DateOffset(months = 6)
last_6M_date

In [ ]:
L6M = df[df['Date'].between(last_6M_date,current_Month_StarDate,inclusive='both')]

In [ ]:
L6M.shape

In [ ]:
L6M=L6M.groupby(by=['Key']).sum()[['Actual','Stat','Consensus','ABS_STAT','ABS_CONS']].reset_index()

In [ ]:
L6M.rename(columns={'Actual':'L6M_Actual_Sum','Stat':'L6M_Stat_Sum','Consensus':'L6M_Consensus_Sum',
                     'ABS_STAT':'L6M_ABSStat_Sum','ABS_CONS':'L6M_ABSCons_Sum'},inplace=True)

In [ ]:
L6M['R6M'] = 'R6M'

### Creating F12M Table

In [ ]:
"""
#F12M_query = textwrap.dedent("""  SELECT [MD_Error]
,[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus]
,sum(cast([Stat]as float)) as [Stat]
,sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [FALossBaseCal]
WHERE [Date] >= Dateadd(Month, -1,getdate())
AND [Date] <= DATEADD(MONTH, 11, CAST(GETDATE() AS DATE))
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin]  """)

"""

In [ ]:
current_Month_StarDate

In [ ]:
forward_12_month = current_Month_StarDate + pd.DateOffset(months=12)

In [ ]:
forward_12_month

In [ ]:
#F12M = pd.read_sql(F12M_query,con=conn)

In [ ]:
F12M = df[df['Date'].between(current_Month_StarDate,forward_12_month,inclusive='both')]

In [ ]:
F12M.shape

In [ ]:
#F12M['Key'] = F12M[['Item','Material Type','Div','Cust Facing Loc','Sales Org','Lag']].agg(','.join, axis=1) 

In [ ]:
#F12M['Key'] = F12M['Item'] +","+ F12M['Material Type'] +","+ F12M['Div'] +","+ F12M['Cust Facing Loc'] +","+ F12M['Sales Org']+","+F12M['Lag']

In [ ]:
F12M = F12M.groupby(['Key']).sum()[['Actual','Stat','Consensus','ABS_STAT','ABS_CONS']].reset_index()

In [ ]:
F12M.rename(columns={'Actual':'F12M_Actual_Sum','Stat':'F12M_Stat_Sum','Consensus':'F12M_Consensus_Sum',
                     'ABS_STAT':'F12M_ABSStat_Sum','ABS_CONS':'F12M_ABSCons_Sum'},inplace=True)

In [ ]:
F12M.shape

### Creating F6M Table

In [ ]:
F6M_query = textwrap.dedent(""" SELECT [MD_Error]
,[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus]
,sum(cast([Stat]as float)) as [Stat]
,sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [FALossBaseCal]
WHERE [Date] >= Dateadd(Month, -1,getdate())
AND [Date] <= DATEADD(MONTH, 5, CAST(GETDATE() AS DATE))
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin] """)

In [ ]:
conn

In [ ]:
#F6M_source = pd.read_sql(F6M_query,con=conn)

In [ ]:
#F6M_source['Key'] = F6M_source['Item'] +","+ F6M_source['Material Type'] +","+ F6M_source['Div'] +","+ F6M_source['Cust Facing Loc'] +","+ F6M_source['Sales Org']+","+F6M_source['Lag']

In [ ]:
#F6M_source['Date'] = pd.to_datetime(F6M_source['Date'])

In [ ]:
current_Month_StarDate

In [ ]:
forward_6Month = current_Month_StarDate + pd.DateOffset(months=6)

In [ ]:
forward_6Month

In [ ]:
F6M = df[df['Date'].between(current_Month_StarDate,forward_6Month,inclusive='both')]

In [ ]:
F6M = pd.DataFrame(F6M.groupby(['Key'])['Actual','Stat','Consensus','ABS_STAT','ABS_CONS'].sum()).reset_index()

In [ ]:
F6M.rename(columns={'Actual':'F6M_Actual_Sum','Stat':'F6M_Stat_Sum','Consensus':'F6M_Consensus_Sum',
                     'ABS_STAT':'F6M_ABSStat_Sum','ABS_CONS':'F6M_ABSCons_Sum'},inplace=True)

In [ ]:
F6M.shape

In [ ]:
F6M.head()

### Creating L12M_Act_count Table

In [ ]:
L12M_act_count_query = textwrap.dedent( """ SELECT [MD_Error],[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus]
,sum(cast([Stat]as float)) as [Stat]
,sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [FABIAS_Dashboard].[FALossBaseCal]
where [Date] > Dateadd(Month, -13, getdate()) and [Date] < Dateadd(Month, -1,getdate())
-- WHERE [Item] = '74700PLUS.61R' and [Lag] = 'Lag 2' and [Period] = '202101'
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin]   """)

In [ ]:
#L12M_Act_Count_source = pd.read_sql(L12M_act_count_query,con=conn)

In [ ]:
#L12M_Act_count = df[df['Date'].between(last_12M_date,current_Month_StarDate,inclusive='both')]

In [ ]:
#L12M_Act_count['Key'] = L12M_Act_count['Item'] +","+ L12M_Act_count['Material Type'] +","+ L12M_Act_count['Div'] +","+ L12M_Act_count['Cust Facing Loc'] +","+ L12M_Act_count['Sales Org']+","+L12M_Act_count['Lag']

In [ ]:
#L12_Act_count = L12M_Act_count['Key'].value_counts().reset_index()

In [ ]:
#L12_Act_count.rename(columns={'index':'Key','Key':'L12M_Act_Count'},inplace=True)

In [ ]:
#L12_Act_count.head()

### Creating L3M Table

In [ ]:
last_3M_date = current_Month_StarDate - pd.DateOffset(months=3)
last_3M_date

In [ ]:
L3M = df[df['Date'].between(last_3M_date,current_Month_StarDate,inclusive='both')]

In [ ]:
L3M = pd.DataFrame(L3M['Key'])

In [ ]:
L3M.reset_index(inplace=True)

In [ ]:
L3M['L3M'] = 'Yes'

In [ ]:
L3M = L3M[['Key','L3M']]

In [ ]:
L3M.shape

In [ ]:
L3M.shape

### Creating L9M

In [ ]:
last_9M_date = current_Month_StarDate - pd.DateOffset(months=9)

In [ ]:
L9M = df[df['Date'].between(last_9M_date,current_Month_StarDate,inclusive='both')]

In [ ]:
L9M.shape

In [ ]:
L9M.reset_index(inplace=True)

In [ ]:
L9M['L9M'] = 'Yes'

In [ ]:
L3M_L9M = pd.merge(L3M,L9M,on='Key',how='right')

In [ ]:
L3M_L9M = pd.DataFrame(data=L3M_L9M[['Key','L3M','L9M']])

In [ ]:
L3M_L9M.shape

### Creating Dashboard Final Data

In [ ]:
"""
dashboard_query = textwrap.dedent( SELECT [MD_Error]
,[Date]
,SUM(cast([ABS_STAT] as float)) AS [ABS_STAT],SUM(cast([ABS_CONS] as float)) AS [ABS_CONS]
,[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc]
,sum(cast([Consensus] as float)) as [Consensus]
,sum(cast([Stat]as float)) as [Stat]
,sum(cast([Actual]as float)) as [Actual]
,[Origin]
FROM [dbo].[FABIAS_Dashboard.FALossBaseCal]
--WHERE [Date] >= Dateadd(Month, -1,getdate())
--AND [Date] <= DATEADD(MONTH, 11, CAST(GETDATE() AS DATE))
GROUP BY
[MD_Error],[Date],[Period],[Lag],[Item],[Material Type],[Div],[Sales Org],[Cust Facing Loc],[Origin])
"""

In [ ]:
#dashboard_df = pd.read_sql(dashboard_query,con=conn)

In [ ]:
print('Source Data Shape is', df.shape)
print('L12M Table Shape is',L12M.shape)
print('L6M Table shape is',L6M.shape)
print('F12M Table Shape is', F12M.shape)
print('F6M Tbale shape is',F6M.shape)
print('L12_Act_count Table shape is', L12_Act_count.shape)
print('L3M Table shape is', L3M.shape)
print('L9M Table shape is ', L9M.shape)
print('L3_L9M Table shape is', L3M_L9M.shape)

In [ ]:
dashboard_df = df[['Date','MD_Error','Key']]

In [ ]:
dashboard_df.shape

In [ ]:
dashboard_df.info()

In [ ]:
dashboard_df.drop_duplicates(inplace=True)

In [ ]:
dashboard_df = pd.merge(dashboard_df,F12M,on='Key',how='outer')

In [ ]:
dashboard_df = pd.merge(dashboard_df,F6M,on='Key',how='outer')

In [ ]:
dashboard_df = pd.merge(dashboard_df,L12M,on='Key',how='outer')

In [ ]:
dashboard_df = pd.merge(dashboard_df,L6M,on='Key',how='outer')

In [ ]:
#dashboard_df = pd.merge(dashboard_df,L12_Act_count,on='Key',how='outer')

In [ ]:
dashboard_df = pd.merge(dashboard_df,L3M_L9M,on='Key',how='outer')

In [ ]:
#dashboard_df.drop(columns='Date_y',inplace=True)

In [ ]:
dashboard_df.head()

In [ ]:
dashboard_df.groupby(['Date'])['L9M'].count()

## Conditions:

In [ ]:
def con_l6m_fa(row):
    if row['L6M_Actual_Sum'] >=1:
        val = row['L6M_ABSStat_Sum']/row['L6M_Actual_Sum']
    else :
        val = np.NaN
    return val

In [ ]:
dashboard_df['L6M_FA'] = dashboard_df.apply(con_l6m_fa,axis=1)

In [ ]:
def con(row):
    if row['L6M_FA'] >= 1:
        val = 0
    elif row['L6M_FA'] < 1:
        val = (1-(row['L6M_FA']))
    else :
        val = np.NaN
    return val

In [ ]:
dashboard_df['L6M_FA_STAT'] = dashboard_df.apply(con,axis=1)

In [ ]:
def l6m_fa_cond(row):
    if row['L6M_Actual_Sum'] >=1 :
        val = row['L6M_ABSCons_Sum']/row['L6M_Actual_Sum']
    else :
        val = np.NaN
    return val

In [ ]:
dashboard_df['L6M_FA_CON'] = dashboard_df.apply(l6m_fa_cond,axis=1)

In [ ]:
def cond2(row):
    if row['L6M_FA_CON'] >= 1:
        val = 0
    elif row['L6M_FA_CON'] < 1:
        val = (1- row['L6M_FA_CON'])
    else:
        val = np.NaN
    return val

In [ ]:
dashboard_df['L6M_FA_CONSEN'] = dashboard_df.apply(cond2,axis=1)

### LEVEL-03 Condition:
    = Table.AddColumn(#"Replaced Errors1", "LEVEL3", each if [MD_Error] = "YES" then "MASTER DATA ERROR" 
    else if [L9M] = "NO" then "TRIAL/NPI" 
    else if [L6M_Actual_Sum] < [L6M_Consensus_Sum] then "OVER FORECASTING" 
    else if [L6M_Actual_Sum] > [L6M_Consensus_Sum] then "UNDER FORECASTING" 
    else if [L6M_Stat_Sum] > [L6M_Consensus_Sum] then "OUTLIER CORRECTION"  
    else if [L12M_Actual_Count] = 0 and [F6M_Consensus_Sum] > 0 then "FORECAST NOT ZEROED OUT" 
    else if [L12M_Actual_Count] < 4 and [L12M_Actual_Count] > 0 then "LUMPY DEMAND" 
    else if [L12M_Actual_Sum] <= 0 and [F6M_Consensus_Sum] <= 0 then "DISCONTINUE" 
    else if [L6M_FA_STAT]>[L6M_FA_CONSEN] then "INCORRECT CHOICE OF MODEL" else "OK/MANUAL INPUT")

In [ ]:
def level3_cond(row):
    if row['MD_Error'] == 'Yes':
        val = 'MASTER DATA ERROR'
    elif row['L9M'] is np.NaN:
        val = 'TRAIL/NPI'
    elif row['L6M_Actual_Sum'] > row['L6M_Consensus_Sum']:
        val = 'UNDER FORECASTING'
    elif row['L6M_Actual_Sum'] < row['L6M_Consensus_Sum']:
        val = "OVER FORECASTING"
    elif row['L6M_Stat_Sum'] > row['L6M_Consensus_Sum']:
        val = "OUTLIER CORRECTION"
    elif row['L12M_Act_Count'] == 0 and row['F6M_Consensus_Sum'] > 0:
        val = "FORECAST NOT ZEROED OUT"
    elif row['L12M_Act_Count'] > 0 and row['L12M_Act_Count'] < 4 :
        val = "LUMPY DEMAND"
    elif row['L12M_Actual_Sum'] <= 0 and row['F6M_Consensus_Sum'] <= 0:
        val = "DISCONTINUE"
    elif row['L6M_FA_STAT'] > row['L6M_FA_CONSEN']:
        val = "INCORRECT CHOICE OF MODEL"
    else:
        val =  "OK/MANUAL INPUT"
    return val

In [ ]:
dashboard_df['LEVEL03'] = dashboard_df.apply(level3_cond,axis=1)

In [ ]:
dashboard_df['LEVEL03'].value_counts()

### LEVEL-02 Condition :

    = Table.AddColumn(#"Added Conditional Column", "LEVEL2", each if [LEVEL3] = "MASTER DATA ERROR" then "MASTER DATA" 
    else if [LEVEL3] = "TRIAL/NPI" then "TRIAL/NPI" 
    else if [LEVEL3] = "OVER FORECASTING" then "FORECASTING ADJUSTMENTS" 
    else if [LEVEL3] = "UNDER FORECASTING" then "FORECASTING ADJUSTMENTS" 
    else if [LEVEL3] = "OUTLIER CORRECTION" then "HISTORY ADJUSTMENTS" 
    else if [LEVEL3] = "FORECAST NOT ZEROED OUT" then "FORECAST MAPPING" 
    else if [LEVEL3] = "LUMPY DEMAND" then "FORECASTING ADJUSTMENTS" 
    else if [LEVEL3] = "DISCONTINUE" then "FORECASTING ADJUSTMENTS" 
    else if [LEVEL3] = "INCORRECT CHOICE OF MODEL" then "FORECASTING MODEL" 
    else if [LEVEL3] = "OK/MANUAL INPUT" then "OK/MANUAL INPUT" 
    else if [L6M_Stat_Sum]=[L6M_Consensus_Sum] and [L6M_FA_CONSEN] <0.75 then "FORECASTING MODEL" 
    else if [L6M_ABSStat_Sum]<>[L6M_ABSCons_Sum] then "FORECASTING ADJUSTMENTS" else "")

In [ ]:
dashboard_df['L9M'].value_counts()

In [ ]:
def level2_condition(row):
    if row['LEVEL03'] == "MASTER DATA ERROR":
        val = "MASTER DATA"
    elif row['LEVEL03'] == "TRIAL/NPI":
        val = "TRIAL/NPI" 
    elif row['LEVEL03'] == "OVER FORECASTING":
        val = "FORECASTING ADJUSTMENTS" 
    elif row['LEVEL03'] == "UNDER FORECASTING":
        val = "FORECASTING ADJUSTMENTS"
    elif row['LEVEL03'] == "OUTLIER CORRECTION":
        val = "HISTORY ADJUSTMENTS"
    elif row['LEVEL03'] == "FORECAST NOT ZEROED OUT":
        val = "FORECAST MAPPING"
    elif row['LEVEL03'] == "LUMPY DEMAND":
        val = "FORECASTING ADJUSTMENTS"
    elif row['LEVEL03'] == "DISCONTINUE":
        val ="FORECASTING ADJUSTMENTS"
    elif row['LEVEL03'] == "INCORRECT CHOICE OF MODEL":
        val = "FORECASTING MODEL"
    elif row['LEVEL03'] == "OK/MANUAL INPUT":
        val = "OK/MANUAL INPUT"
    elif row['L6M_Stat_Sum']==row['L6M_Consensus_Sum'] and row['L6M_FA_CONSEN'] < 0.75 :
        val = "FORECASTING MODEL"
    elif row['L6M_ABSStat_Sum'] != row['L6M_ABSCons_Sum']:
        val = 'FORECASTING ADJUSTMENTS'
    else:
        val = 'NA'
    return val

In [ ]:
dashboard_df['LEVEL02'] = dashboard_df.apply(level2_condition,axis=1)

In [ ]:
px.pie(dashboard_df,names='LEVEL02',width=800, height=400)

### Level1 Condition :
    """
    Added Custom2", "LEVEL1", each if [LEVEL2] = "MASTER DATA" then "DATA" 
    else if [LEVEL2]="TRIAL/NPI" then "SALES MARKETING INPUT" 
    else if [LEVEL2] = "FORECASTING ADJUSTMENTS" then "FORECAST PROCESS" 
    else if [LEVEL2] = "HISTORY ADJUSTMENTS" then "FORECAST PROCESS" 
    else if [LEVEL2] = "FORECAST MAPPING" then "FORECAST PROCESS" 
    else if [LEVEL2] = "FORECASTING MODEL" then "FORECAST PROCESS" 
    else if [LEVEL2] = "OK/MANUAL INPUT" then "OK/MANUAL INPUT" else "NA"
    """

In [ ]:
def level01_condition(row):
    if row['LEVEL02'] == "MASTER DATA":
        val = "DATA"
    elif row['LEVEL02'] == "TRIAL/NPI":
        val = "SALES MARKETING INPUT"
    elif row['LEVEL02'] == "FORECASTING ADJUSTMENTS":
        val = "FORECAST PROCESS"
    elif row['LEVEL02'] == "HISTORY ADJUSTMENTS":
        val = "FORECAST PROCESS"
    elif row['LEVEL02'] == "FORECAST MAPPING":
        val = "FORECAST PROCESS"
    elif row['LEVEL02'] == "FORECASTING MODEL":
        val = "FORECAST PROCESS"
    elif row['LEVEL02'] == "OK/MANUAL INPUT":
        val = "OK/MANUAL INPUT"
    else:
        val = 'NA'
    return val

In [ ]:
px.pie(dashboard_df,names='LEVEL01',width=800, height=400)

In [ ]:
dashboard_df['LEVEL01'] = dashboard_df.apply(level01_condition,axis=1)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=1,cols=3)
fig.add_trace(gdashboard_df,names='LEVEL01',width=800, height=400)
fig.show()

In [ ]:
#fig = px.pie(data_frame=dashboard_df,names='LEVEL02', width=500,height=200)
#fig.update_layout(margin=dict(l=20, r=20, t=20, b=20),paper_bgcolor="LightSteelBlue")
#fig.show()

In [ ]:
dashboard_df[['Item','Material_Type','Division','Cust_Facing_Loc','Sales_org','Lag']] = dashboard_df['Key'].str.split(',',5,expand=True)

In [ ]:
dashboard_df['Abs_Error_Cons'] = dashboard_df['L12M_Actual_Sum'] - dashboard_df['L12M_Consensus_Sum']

In [ ]:
dashboard_df['Abs_Error_Cons'] = dashboard_df['Abs_Error_Cons'].apply(lambda x:abs(x))

### FA Calculation :

* Formula : R12M_FA_CONS = IFERROR(IF((SUM(DASHBOARD[L12M_ABSERROR_CONS])/SUM(L12M[L12M_Actual_Sum]))>1,
                          0,1-(SUM(DASHBOARD[L12M_ABSERROR_CONS])/SUM(DASHBOARD[L12M_Actual_Sum]))),BLANK())


In [ ]:
def FA(row):
    if row['L12M_Actual_Sum'] == 0:
        val = np.NaN
    elif (row['Abs_Error_Cons']/row['L12M_Actual_Sum']) >1:
        val = 0
    elif (row['Abs_Error_Cons']/row['L12M_Actual_Sum']) <=1:
        val = (1-(row['Abs_Error_Cons']/row['L12M_Actual_Sum']))
    else :
        val = np.NaN
    return val

In [ ]:
dashboard_df['FA'] = dashboard_df.apply(FA,axis=1)

### BAIS Calculalation :

#### Formula :

* L12M_BIAS_CONSEN = IFERROR(((SUM(DASHBOARD[L12M_Actual_Sum])-   SUM(DASHBOARD[L12M_Consensus_Sum]))/SUM(DASHBOARD[L12M_Consensus_Sum]))*-1,0) 

In [ ]:
def bais_cond(row):
    if row['L12M_Consensus_Sum'] > 0:
        val = (row['L12M_Consensus_Sum']-row['L12M_Actual_Sum'])/row['L12M_Consensus_Sum']
    else :
        val = np.NaN
    return val

In [ ]:
dashboard_df['Bais'] = dashboard_df.apply(bais_cond,axis=1)

In [ ]:
dashboard_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
loss_df = dashboard_df[['Date','Item','LEVEL01','LEVEL02','LEVEL03','Division','Material_Type','Lag','Sales_org','Cust_Facing_Loc']]

In [ ]:
loss_df['Forecast_Consensus'] = dashboard_df['L12M_Consensus_Sum']

In [ ]:
loss_df['Actual'] = dashboard_df['L12M_Actual_Sum']

In [ ]:
loss_df['Abs_Error_Cons'] = dashboard_df['L12M_Actual_Sum'] - dashboard_df['L12M_Consensus_Sum']

In [ ]:
loss_df['Abs_Error_Cons'] = loss_df['Abs_Error_Cons'].apply(lambda x:abs(x))

In [ ]:
loss_df['FA'] = dashboard_df['FA']
loss_df['BAIS'] = dashboard_df['Bais']

In [ ]:
#loss_df.to_excel('loss_df.xlsx',index=False)

In [ ]:
#dashboard_df.to_excel('Dashboard_df.xlsx',index=False)

In [ ]:
loss_df.shape

In [ ]:
loss_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
f, axes = plt.subplots(1, 3, figsize=(15, 7), sharex=True)            # Set up the matplotlib figure
sns.despine(left=False)
loss_df['LEVEL01'].value_counts().plot(kind='pie',ax=axes[0])
loss_df['LEVEL02'].value_counts().plot(kind='pie',ax=axes[1])
loss_df['LEVEL03'].value_counts().plot(kind='pie',ax=axes[2])

In [ ]:
cust_funnel_dat=pd.DataFrame(data=loss_df.groupby(['Cust_Facing_Loc']).sum()[['Abs_Error_Cons']].nlargest(10,columns='Abs_Error_Cons'))

In [ ]:
cust_funnel_dat.reset_index(inplace=True)

In [ ]:
loss_df['Date'].min()

In [ ]:
px.funnel(cust_funnel_dat, y='Cust_Facing_Loc', x='Abs_Error_Cons')

In [ ]:
sales_funnel=pd.DataFrame(loss_df.groupby(['Sales_org']).sum()[['Abs_Error_Cons']].nlargest(10,columns='Abs_Error_Cons'))

In [ ]:
sales_funnel.reset_index(inplace=True)

In [ ]:
px.funnel(sales_funnel, y='Sales_org', x='Abs_Error_Cons')

In [ ]:
sku_funnel=pd.DataFrame(loss_df.groupby(['Item']).sum()[['Abs_Error_Cons']].nlargest(10,columns='Abs_Error_Cons'))

In [ ]:
sku_funnel.reset_index(inplace=True)

In [ ]:
px.funnel(sku_funnel, y='Item', x='Abs_Error_Cons')